In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [104]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [5]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [6]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [7]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [129]:
df_1 = pd.merge(left=profile, right=transcript, left_on='id', right_on='person')

In [130]:
df_1 = df_1.drop('person', axis=1)

In [131]:
df_1.value[4]

{'amount': 0.35000000000000003}

In [135]:
def find_offer_id(row):
    if 'offer id' in row.keys():
        label = row['offer id']
    else: label = np.NaN
    return label
def find_amount(row):
    if 'amount' in row.keys():
        label = row['amount']
    else: label = np.NaN
    return label

In [133]:
df_1['offer_id'] = df_1.value.apply(lambda x: find_offer_id(x))

In [136]:
df_1['amount'] = df_1.value.apply(lambda x: find_amount(x))

In [137]:
df_2 = pd.merge(left=df_1, right=portfolio, left_on='offer_id', right_on='id', suffixes=('_customer','_offer'))

In [139]:
df_2.drop('value', axis=1, inplace=True)

In [140]:
df_2.head(10)

,age,became_member_on,gender,id_customer,income,event,time,offer_id,amount,channels,difficulty,duration,id_offer,offer_type,reward
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer received,168,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
1,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,offer viewed,216,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
2,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
3,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0,offer viewed,18,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
4,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,offer received,408,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
5,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,offer viewed,462,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
6,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN,offer received,408,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN,offer viewed,432,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0,offer received,336,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
9,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0,offer viewed,354,2906b810c7d4411798c6938adc9daaa5,NaN,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2
